# Development Notebooks

## Create Points for StreamStats

Usage Notes:


Outputs:


by Shane Putnam: sputnam@dewberry.com & Seth Lawler: slawler@dewberry.com

In [1]:
import os
from osgeo import gdal, ogr,osr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from StreamStats_Points import*

### Load the masked stream grid:

In [2]:
in_tif=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was masked by the catchment polygon

In [3]:
sg = StreamGrid(in_tif) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) #Print the value

epsg: 5070


In [4]:
df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Specify the pour point to set the start location of the search:

In [5]:
lat=1925315.186 #latitude of the pourpoint at the catchment outlet
lon=1616784.964 #longitude of the pourpoint

In [6]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print(pourpoint)

[(877, 1848)]


### Move up the stream and identify the confluences:

In [7]:
nogo=[] #Empty list to store the nogo points, i.e. the stream cells we previously searched
ppoints=[] #Empty list to store the cells which are initially identified as confluences

cnum=0 #The confluence number, start it at zero

confl=[pourpoint[0]+ (cnum,)] #Add the confluence number to the original pourpoint touple

In [8]:
l=len(confl)-1
i=0
while i<=l:
    confl1=[]
    confl1, nogo, ppoints=NextConfluence(df, confl[i], nogo, ppoints)
    if len(confl1)==0:
        i+=1
    elif len(confl1)>0:
        confl=confl+confl1
        l=len(confl)-1
        i+=1

In [9]:
b=[]
for i in range(len(ppoints)):
    b.append(ppoints[i][2])    

import collections
a=collections.Counter(b)

conflIDs=[]
for i in list(a.keys()):
    if a[i]>1:
        conflIDs.append(i)
        
ppointsqc=[]
for i in range(len(ppoints)):
    if ppoints[i][2] in conflIDs:
        ppointsqc.append(ppoints[i])   

### Save the conflunces as a shapefile:

In [10]:
longitude, latitude=index2coord(sg, ppointsqc)

In [11]:
gdf=geodataframe(longitude, latitude, crs)

gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\allconfluences.shp')

gdf.head(2)

C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


,Lon,Lat,Coordinates
0,1616725.0,1925775.0,POINT (1616725 1925775)
1,1616775.0,1925795.0,POINT (1616775 1925795)


In [12]:
len(ppointsqc)

470